<img width="8%" alt="News API.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/News%20API.png" style="border-radius: 15%">

# News API - Get top headlines

**Tags:** #newsapi #python #api #headlines #news #data #command

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2024-02-07 (Created: 2024-02-07)

**Description:** This notebook fetches the top headlines from News API.

**References:**
- [Get Your NewsAPI key](https://newsapi.org/register).
- [News API Documentation](https://newsapi.org/docs/client-libraries/python)
- [Python Requests Library](https://docs.python-requests.org/en/latest/)

## Input

### Import libraries

In [ ]:
import naas
import requests

### Setup variables
- `keywords`: naas.ai, Data & AI platform
- `api_key`: Your News API key.
- `body`: This variable stores the body to be send by the webhook.

In [ ]:
keywords = "Microsoft"
api_key = None
body = {}

### Setup parameters
The webhook body will be injected below this cell when the webhook is triggered. 
Therefore, it is important to set up how you will handle the injected variable from the body in order to make your script work.
To receive the body from the webhook, please ensure that this cell is tagged as "parameters".

In [ ]:
# Parameters
if len(body) > 0:
    keywords = body.get("keywords")
    api_key = body.get("api_key")

## Model

### Get News

In [ ]:
def fetch_top_headlines(api_key, keywords):
    # Init
    status = "ok"
    
    # Manage api key
    if api_key is None or str(api_key) == "":
        if naas.secret.get("NEWS_API_API_KEY") is None:
            message = f"❌ Unable to Connect to News API. Please update your API Key or your keywords: '{keywords}'"
            return status, message
        else:
            api_key = naas.secret.get("NEWS_API_API_KEY")
    
    # Requests
    try:
        url = f"https://newsapi.org/v2/top-headlines?q={keywords}"
        headers = {"Authorization": f"Bearer {api_key}"}
        res = requests.get(url, headers=headers)
        if len(res.json()) == 0 or len(res.json().get('articles')) == 0:
            message = f"✅ No articles found."
        else:
            message = f"""
            Please provide the keywords result '✅ We have found {len(res.json().get('articles'))} articles on {keywords} from the News API.' and give a concise summary of the first 3 articles, sorted by date. If there is more than 3 articles, ask if the user wants to know more about the other articles. Start with the 'publishedAt' date in ISO format, followed by the 'title' and 'url' in markdown format. Then, provide a brief description in English. Display the associated image using markdown as 'followImage'. Conclude with the author's name and the source name below the image. Each article should be presented as a bullet point in a list without creating sub-bullet points for key elements of each article.
            Articles: 
            {res.json().get('articles')}
            """
        return status, message
    except Exception as e:
        message = f"❌ Unable to Connect to News API. Error message: '{e}'"
        return status, message
        
status, message = fetch_top_headlines(api_key, keywords)
print("Status:", status)
print("Message:", message)

## Output

### Create or update Webhook

In [ ]:
naas.webhook.add()

### Return JSON response
Response sent to the browser before displayed in Chat UI.

In [ ]:
naas.webhook.respond_json(
    {
        "status": status, 
        "message": message
    }
)